In [ ]:
#install HERE!

In [1]:
#import HERE!
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
#retrieve the challenge files from google drive, using a git repositories generates bugs if u restart the runtime -.-"
#if u haven't downloaded them yet, see: https://halmstaduniversity.app.box.com/s/x2e8gfcb37an77wwc566ror3r3sg2yg1
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
#can find your path going on the "folder image" on the left,
#navigating your drive till the project folder, then selecting "copy path" in the "3dots menu" u.u
DIR_PATH = '/content/drive/MyDrive/MaLTo/Volvo Challenge ECML 2024'

In [16]:
#just a check u're not messin up: u should see the list of your folder's descendants O.ò
!ls {DIR_PATH.replace(" ","\ ")}

 public_X_test.csv   startkit.py      variants.csv
 SampleSubmission    train_gen1.csv  'Volvo Challenge Description.pdf'


In [19]:
#try the demo steps from "startkit.py"...
#notice that I added my DIR_PATH wrt the original code °u°
#u should see as output the shape of our data

#they also used a seed for rng
np.random.seed(40)

df_train = pd.read_csv(os.path.join(DIR_PATH,'train_gen1.csv'))
X_test = pd.read_csv(os.path.join(DIR_PATH,'public_X_test.csv'))
df_variants = pd.read_csv(os.path.join(DIR_PATH,'variants.csv'))

print('Subsampling the training data to speed up the training process ...')
df_train = df_train.sample(frac=0.2)

print('df_train.shape:', df_train.shape)
print('X_test.shape:', X_test.shape)
print('df_variants.shape:', df_variants.shape)

Subsampling the training data to speed up the training process ...
df_train.shape: (31487, 308)
X_test.shape: (33590, 307)
df_variants.shape: (10639, 13)


In [20]:
#they decided to impute missing and to always set to 0...we should check this O.ò
# impute missing values
df_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [22]:
#They fitted a tree
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(random_state=42)

In [23]:
#dropping columns and getting train X and y
X_train = df_train.drop(columns=['Timesteps', 'ChassisId_encoded', 'gen', 'risk_level'])
y_train = df_train['risk_level']

In [36]:
#notice that our labels are still strings and not in one-hot encoding °u°
y_train.iloc[0], y_train.shape, y_train.name, (y_train.unique())

('Low', (31487,), 'risk_level', array(['Low', 'Medium', 'High'], dtype=object))

In [37]:
#fitting the model...
print('fitting the classifier ...')
classifier.fit(X_train, y_train)

fitting the classifier ...


DecisionTreeClassifier(random_state=42)

In [38]:
#predicting the test set...
y_pred = classifier.predict(X_test.drop(columns=['Timesteps', 'ChassisId_encoded', 'gen']))

In [39]:
#saving my predictions: I suppose it'll be the required format
#again, notice that I added my DIR_PATH wrt to the original code
df_pred = pd.DataFrame(data=y_pred, columns=['pred'])
df_pred.to_csv(os.path.join(DIR_PATH,'SampleSubmission/prediction.csv'), index=False)

In [40]:
#here they create a placeholder for the test set ground truth O.O'
#by putting in loop one of the three labels
#so don't worry if what follows is bad

# evaluating the prediction against a mocked up ground truth
length = len(X_test)
sequence = np.array(['Low', 'Medium', 'High'])
mocked_true = sequence[np.arange(length) % len(sequence)]

In [46]:
#they do this to make their life easier and mine harder :Q___
#they are just putting what they'll need for their F1 score together
y_true = pd.DataFrame()
y_true['risk_level'] = mocked_true
y_true['pred'] = y_pred
y_true['gen'] = X_test['gen']

In [54]:
#just see if it's all ok...
#Don't ask me what gen are
y_true.iloc[0], y_true['gen'].unique()

(risk_level     Low
 pred           Low
 gen           gen2
 Name: 0, dtype: object,
 array(['gen2', 'gen1'], dtype=object))

In [47]:
#their function for the F1 score: I wouldn't change a single char...
from sklearn.metrics import precision_recall_fscore_support
def macro_avg_f1(y_true, y_pred):
    # Get precision, recall, and f1-score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred)

    # Calculate macro average F1-score
    macro_avg_f1 = sum(f1) / len(f1)

    return round(macro_avg_f1, 2)

In [48]:
#compute F1 score and create a dictionary
#again, it's not a mandatory step for the submission
print('*** Checking Performance of the model agianst mocked up truth labels ***')
score1 = macro_avg_f1(y_true[y_true['gen'] == 'gen1']['risk_level'],
                            y_true[y_true['gen'] == 'gen1']['pred'])
score2 = macro_avg_f1(y_true[y_true['gen'] == 'gen2']['risk_level'],
                            y_true[y_true['gen'] == 'gen2']['pred'])

scores = {
    'Final Score': round((score1 + score2)/2, 2),
    'score on gen1:': score1,
    'score on gen2:': score2
}

*** Checking Performance of the model agianst mocked up truth labels ***


In [50]:
#print the results (don't panic if you are using a mocked truth)
print(f'Evaluation results against mocked up ground truth: {macro_avg_f1(mocked_true, y_pred)}')
print('\nScores Details:\n', scores)

Evaluation results against mocked up ground truth: 0.21

Scores Details:
 {'Final Score': 0.21, 'score on gen1:': 0.24, 'score on gen2:': 0.18}


In [51]:
#this is the last step to create the .zip for the submission *u*
import zipfile
def compress_file(input_file, output_zip):
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(input_file, os.path.basename(input_file))

In [53]:
#again, notice that I added my DIR_PATH wrt to the original code
input_file = os.path.join(DIR_PATH,'SampleSubmission/prediction.csv')  # Input file to compress
output_zip = os.path.join(DIR_PATH,'SampleSubmission/prediction.csv.zip')  # Output ZIP archive
compress_file(input_file, output_zip)
print(f'File "{output_zip}" successfully created for submission to the Codabench portal.')

File "/content/drive/MyDrive/MaLTo/Volvo Challenge ECML 2024/SampleSubmission/prediction.csv.zip" successfully created for submission to the Codabench portal.
